In [12]:
import csv
import os
import time

from scipy import *
import numpy as np

from srtm import get_elevation
from srtm import get_file_name
from srtm import load_elevations_into_memory

def append_csv(output_file, data):
    try:
        with open(output_file, 'a') as file:
            writer = csv.writer(file)
            writer.writerow(data)
    except (OSError, IOError) as e:
        print e

In [13]:
#input SRTM files
srtm_folder = 'britain-srtm-30m'

In [56]:
#output file
samples = 3601
meter_distance = 450
degree_sep = meter_distance / 30. / 3601.

minlat = 51.
maxlat = 55.
minlon = -11.
maxlon = -6.

# create file name
area_name = 'areaData_30m_'
if minlat >= 0:
    area_name += 'N%02d' % abs(minlat)
    area_name += 'N%02d' % abs(maxlat)
else:
    area_name += 'S%02d' % abs(minlat)
    if maxlat <= 0 :
        area_name += 'S%02d' % abs(maxlat)
    else :
        area_name += 'N%02d' % abs(maxlat)
area_name += '_'
if minlon >= 0:
    area_name += 'E%02d' % abs(minlon)
    area_name += 'E%02d' % abs(maxlon)
else:
    area_name += 'W%02d' % abs(minlon)
    if maxlon <= 0 :
        area_name += 'W%02d' % abs(maxlon)
    else :
        area_name += 'E%02d' % abs(maxlon) 
area_name += '.csv'

area_name_folder = ''
area_name = os.path.join(area_name_folder, area_name)

print area_name

# make folder path
if area_name_folder and not os.path.isdir(area_name_folder):
    os.mkdir(area_name_folder)

areaData_30m_N51N55_W11W06.csv


In [60]:
t0 = time.time()

with open(area_name, 'w') as areaDataFile:
    
    i=0
    for lon in arange(minlon, maxlon+1):
        for lat in arange(minlat, maxlat+1):

            # load elevation data into memory
            srtm_file_name = get_file_name(lon, lat, srtm_folder)
            if srtm_file_name == None:
                elevation_loaded = False
            else:
                elevation_loaded = True
                elevations = load_elevations_into_memory(srtm_file_name, lon, lat)
                print 'loaded %s' % srtm_file_name

                head = '%s,%s,%s,%s\n' % ('id','Latitude', 'Longitude', 'Elevation')
                areaDataFile.write(head)

                for lon_i in arange(lon, lon+1., degree_sep):

                    lon_row = 0 
                    if elevation_loaded:
                        lon_row = int(round((lon_i - int(lon_i)) * (samples - 1), 0))

                    for lat_i in arange(lat, lat+1., degree_sep):

                        if not elevation_loaded:
                            srtm_file_name = get_file_name(lon, lat, srtm_folder)
                            if srtm_file_name == None:
                                elevation_loaded = False
                            else:
                                elevation_loaded = True
                                elevations = load_elevations_into_memory(srtm_file_name, lon, lat)
                                lon_row = int(round((lon_i - int(lon_i)) * (samples - 1), 0))
                                print 'loaded %s' % srtm_file_name
                        
                        # remove great britain points
                        if lon_i > -5.50003                     \
                           or (lon_i > -5.1 and lat_i > 55)     \
                           or (lon_i > -6.68 and lat_i > 55.47) \
                           or (lon_i > -5.91 and lat_i > 55.16):
                            continue
                        
                        lat_row = int(round((lat_i - int(lat_i)) * (samples - 1), 0))
                        elevation = elevations[samples - 1 - lat_row, lon_row].astype(int)
                        data = '%d,%.4f,%.4f,%d\n' % (i, lat_i, lon_i, elevation)
                        if elevation > 0:
                            areaDataFile.write(data)
                            i += 1
tf = time.time()

print 'Total time for %d meter separation: %f seconds' % (meter_distance, tf-t0)
print 'Number of elements %d' % i

loaded britain-srtm-30m/N51W011.hgt
loaded britain-srtm-30m/N52W011.hgt
loaded britain-srtm-30m/N53W011.hgt
loaded britain-srtm-30m/N54W011.hgt
loaded britain-srtm-30m/N51W010.hgt
loaded britain-srtm-30m/N52W010.hgt
loaded britain-srtm-30m/N53W010.hgt
loaded britain-srtm-30m/N54W010.hgt
loaded britain-srtm-30m/N51W009.hgt
loaded britain-srtm-30m/N52W009.hgt
loaded britain-srtm-30m/N53W009.hgt
loaded britain-srtm-30m/N54W009.hgt
loaded britain-srtm-30m/N55W009.hgt
loaded britain-srtm-30m/N51W008.hgt
loaded britain-srtm-30m/N52W008.hgt
loaded britain-srtm-30m/N53W008.hgt
loaded britain-srtm-30m/N54W008.hgt
loaded britain-srtm-30m/N55W008.hgt
loaded britain-srtm-30m/N52W007.hgt
loaded britain-srtm-30m/N53W007.hgt
loaded britain-srtm-30m/N54W007.hgt
loaded britain-srtm-30m/N55W007.hgt
loaded britain-srtm-30m/N51W006.hgt
loaded britain-srtm-30m/N52W006.hgt
loaded britain-srtm-30m/N53W006.hgt
loaded britain-srtm-30m/N54W006.hgt
loaded britain-srtm-30m/N55W006.hgt
Total time for 450 meter sep

In [55]:
print get_elevation(51.6055,-9.4614, srtm_folder)

-32768
